In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm

# Discriminator Architecture (Pix2Pix)

The `Discriminator` class implements a **PatchGAN-based** discriminator, commonly used in the **Pix2Pix** model for image-to-image translation. It classifies whether each patch in an image is real or fake.

## Architecture Overview

- The discriminator takes as input a **pair of images** `(x, y)`, where:
  - `x` is the input image.
  - `y` is the target/generated image.
  - These two images are concatenated along the channel dimension (`dim=1`).
  
- It consists of:
  1. **An Initial Convolution Layer**:
     - Uses a `Conv2d` layer with:
       - Input channels: `in_channels * 2` (since both `x` and `y` are concatenated).
       - Output channels: `features[0]` (typically `64`).
       - Kernel size: `4x4`.
       - Stride: `2` (reduces spatial dimensions).
       - No Batch Normalization (as in the Pix2Pix paper).
       - Activation: `LeakyReLU(0.2)`.
  
  2. **A Series of CNN Blocks (`CNNBlock`)**:
     - Each block consists of:
       - A `Conv2d` layer with **Batch Normalization** and **LeakyReLU(0.2)**.
       - Stride is `2` for all layers **except** the last one.
       - Channels progress as defined in `features=[64, 128, 256, 512]`.
  
  3. **Final Convolution Layer**:
     - A single **`Conv2d`** layer with:
       - Output channels = `1` (discriminator outputs a single-channel probability map).
       - Kernel size: `4x4`.
       - Stride: `1`.
       - Padding: `1`.
       - No activation function (raw logits output).

## Notes

- The discriminator **does not output a single value** but rather a **feature map**, where each patch's value represents its real/fake probability.
- The `padding_mode="reflect"` is used to **reduce artifacts**.

In [2]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False, padding_mode="reflect"),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.conv(x)
        
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()

        self.initial = nn.Sequential(
            # Channels are doubled because in Pix2Pix model, the discriminator takes (x,y) pair as input
            # Initial layer does not perform batch normalization and hence specified separately
            nn.Conv2d(in_channels*2, features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        convolution_layers = []
        in_channels = features[0]
        
        for feature in features[1:]:
            convolution_layers.append(
                # Stride is 2 except for the last layer
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            )
            in_channels = feature

        convolution_layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            )
        )
        
        self.model = nn.Sequential(*convolution_layers)

    def forward(self, x, y):
        # Concatenate x and y along the channel
        x = torch.cat([x,y], dim=1)
        x = self.initial(x)
        return self.model(x)

# Generator Architecture (UNet-based)

The `Generator` class implements a **UNet-based** architecture, commonly used in **Pix2Pix** for image-to-image translation. It follows an **encoder-decoder** structure with **skip connections** to retain spatial information lost during downsampling.

## Architecture Overview

The generator consists of the following components:

1. **Initial Downsampling Block**:
   - A `Conv2d` layer with:
     - Input channels: `in_channels` (default: `3` for RGB).
     - Output channels: `features` (default: `64`).
     - Kernel size: `4x4`, stride `2`, padding `1`.
     - `LeakyReLU(0.2)` activation.
   - No batch normalization is applied in this layer.

2. **Series of Encoder Blocks**:
   - Each encoder block (`UNetBlock`) performs **downsampling** using:
     - A `Conv2d` layer with `stride=2` to reduce spatial dimensions.
     - Batch Normalization.
     - `LeakyReLU(0.2)` activation.
   - Skip connections store intermediate outputs for later use in the decoder.

3. **Bottleneck Layer**:
   - A `Conv2d` layer with:
     - Kernel size: `4x4`, stride `2`, padding `1`.
     - `ReLU` activation (no batch normalization).
   - A **bottleneck upsampling** block follows, performing:
     - Transposed convolution (`ConvTranspose2d`).
     - `ReLU` activation.
     - **Dropout (`0.5`)** is applied to this layer.

4. **Series of Decoder Blocks**:
   - Each decoder block (`UNetBlock`) performs **upsampling** using:
     - A `ConvTranspose2d` layer with `stride=2` to increase spatial dimensions.
     - Batch Normalization.
     - `ReLU` activation.
     - **Dropout** applied for the first `decoder_dropout_range` layers.
   - Decoder blocks take input from both:
     - The previous decoder layer.
     - The **corresponding encoder output** (via **skip connections**).

5. **Final Upsampling Block**:
   - A `ConvTranspose2d` layer with:
     - Output channels: `in_channels` (matching input size).
     - `Tanh` activation to normalize output values between `[-1, 1]`.

## Notes

- **Skip Connections** help retain high-resolution details lost during downsampling.
- The **use of dropout** in certain decoder layers acts as regularization.
- The generator **learns a mapping** from input images (`x`) to target images (`y`), making it suitable for **image-to-image translation**.

In [3]:
class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down_sample=True, activation="relu", use_dropout=False):
        super().__init__()

        layers = []
        if down_sample:
            layers.append(nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect"))
        else:
            layers.append(nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False))
            
        layers.append(nn.BatchNorm2d(out_channels))

        if activation == "relu":
            layers.append(nn.ReLU())
        else:
            layers.append(nn.LeakyReLU(0.2))
            
        if use_dropout:
            layers.append(nn.Dropout(0.5))
            
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        return self.conv(x)
        
class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64, encoder_blocks=[], decoder_blocks=[], decoder_dropout_range=0):
        super().__init__()

        # Initial downward block with no batch normalization
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2)
        )

        # Series of downsampling encoders
        self.encoders = nn.ModuleList()
        for i, o in encoder_blocks:
            self.encoders.append(
                UNetBlock(features*i, features*o, down_sample=True, activation="leaky", use_dropout=False)
            )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, bias=False),
            nn.ReLU()
        )

        self.bottleneck_up = nn.Sequential(
            UNetBlock(features*8, features*8, down_sample=False, activation="relu", use_dropout=True)
        )

        # Series of upsampling decoders
        self.decoders = nn.ModuleList()
        for idx, (i, o) in enumerate(decoder_blocks):
            use_dropout = idx < decoder_dropout_range
            self.decoders.append(
                # Multiplied by two to account for skip connection concatenation
                UNetBlock(features*i*2, features*o, down_sample=False, activation="relu", use_dropout=use_dropout)
            )

        # Instead of one final layer, we create two branches:
        # One for generating the nasal image and one for the temporal image.
        self.final_nasal = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, 4, 2, 1),
            nn.Tanh()
        )
        self.final_temporal = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        skip_connections = []
        d = self.initial(x)
        skip_connections.append(d)

        for encoder in self.encoders:
            d = encoder(d)
            skip_connections.append(d)

        bottleneck = self.bottleneck(d)
        u = self.bottleneck_up(bottleneck)

        # Reverse the skip connections to match encoder-decoder
        for decoder, skip_encoder in zip(self.decoders, list(reversed(skip_connections))[:-1]):
            u = decoder(torch.cat([u, skip_encoder], dim=1))

        # Concatenate top level skip connection with decoder output
        features = torch.cat([u, skip_connections[0]], dim=1)
        
        # Generate the two outputs using separate final layers
        nasal_output = self.final_nasal(features)
        temporal_output = self.final_temporal(features)

        # Connect top level encoder with decoder
        return nasal_output, temporal_output

In [4]:
class RetinaDataSet(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.center_dir = root_dir + '/Center'
        self.nasal_dir = root_dir + '/Nasal'
        self.temporal_dir = root_dir + '/Temporal'
        
        self.center_file_lists = os.listdir(self.center_dir)
        self.nasal_file_lists = os.listdir(self.nasal_dir)
        self.temporal_file_lists = os.listdir(self.temporal_dir)
        
        self.transformer = transforms.ToTensor()

    def __len__(self):
        return len(self.center_file_lists)

    def __getitem__(self, index):
        # Load center image
        center_img_file = self.center_file_lists[index]
        center_img_path = os.path.join(self.center_dir, center_img_file)
        center_img = Image.open(center_img_path)
        center_img_tensor = self.transformer(center_img)
        
        # Load nasal image
        nasal_img_file = self.nasal_file_lists[index]
        nasal_img_path = os.path.join(self.nasal_dir, nasal_img_file)
        nasal_img = Image.open(nasal_img_path)
        nasal_img_tensor = self.transformer(nasal_img)
        
        # Load temporal image
        temporal_img_file = self.temporal_file_lists[index]
        temporal_img_path = os.path.join(self.temporal_dir, temporal_img_file)
        temporal_img = Image.open(temporal_img_path)
        temporal_img_tensor = self.transformer(temporal_img)
    
        # Define and apply resize transformation
        resize_transform = transforms.Compose([
            transforms.Resize((256, 256))
        ])
        img_center_resized = resize_transform(center_img_tensor)
        img_nasal_resized = resize_transform(nasal_img_tensor)
        img_temporal_resized = resize_transform(temporal_img_tensor)
    
        # Return center image as input and both nasal and temporal as targets
        return img_center_resized, (img_nasal_resized, img_temporal_resized)

In [5]:
def save_some_examples(gen, disc, val_loader, epoch, best_val, l1_lambda, folder):
    # Get a single batch from the validation set
    # Here, x is the center image and targets is a tuple: (nasal_target, temporal_target)
    x, targets = next(iter(val_loader))
    x = x.to(device).float()
    nasal_target = targets[0].to(device).float()
    temporal_target = targets[1].to(device).float()

    # Set both generator and discriminator to evaluation mode
    gen.eval()
    disc.eval()

    with torch.no_grad():
        # Generate the fake images: nasal and temporal outputs from the same center input
        nasal_fake, temporal_fake = gen(x)

        # Save images for visual comparison
        save_image(nasal_fake, folder + f"/fake_nasal_{epoch}.png")
        save_image(temporal_fake, folder + f"/fake_temporal_{epoch}.png")
        save_image(x, folder + f"/real_center_{epoch}.png")
        save_image(nasal_target, folder + f"/target_nasal_{epoch}.png")
        save_image(temporal_target, folder + f"/target_temporal_{epoch}.png")

        # Calculate generator validation loss for the nasal branch
        d_nasal_fake = disc(x, nasal_fake)
        g_nasal_fake_loss = BCE_loss(d_nasal_fake, torch.ones_like(d_nasal_fake))
        l1_nasal = L1_loss(nasal_fake, nasal_target) * l1_lambda

        # Calculate generator validation loss for the temporal branch
        d_temporal_fake = disc(x, temporal_fake)
        g_temporal_fake_loss = BCE_loss(d_temporal_fake, torch.ones_like(d_temporal_fake))
        l1_temporal = L1_loss(temporal_fake, temporal_target) * l1_lambda

        # Total generator loss is the sum of the losses from both branches
        g_loss = g_nasal_fake_loss + l1_nasal + g_temporal_fake_loss + l1_temporal

        if g_loss < best_val:
            best_val = g_loss
            save_checkpoint(disc, disc_opt, filename="./SavedModel/c_to_nt/c_to_nt_disc_" + str(epoch) + ".pth.tar")
            save_checkpoint(gen, gen_opt, filename="./SavedModel/c_to_nt/c_to_nt_gen_" + str(epoch) + ".pth.tar")
        print(f"Epoch {epoch} Validation Generator Loss: {g_loss.item():.4f}")

    # Switch back to training mode
    gen.train()
    disc.train()

    return best_val, g_loss


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_file, model, optimizer, lr, device):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train_file_dir = "./dataset/retinal/train"
val_file_dir = "./dataset/retinal/val"
lr = 2e-4
batch_size = 16
img_size = 256
img_channels = 3
l1_lambda = 100
lambda_gp = 10
num_epochs = 6001

disc = Discriminator(
    in_channels=3,
    features=[64, 128, 256, 512]
).to(device)
gen = Generator(
    in_channels=3,
    features=64,
    encoder_blocks=[(1,2), (2,4), (4,8), (8,8), (8,8), (8,8)],
    decoder_blocks=[(8,8), (8,8), (8,8), (8,4), (4,2), (2,1)],
    decoder_dropout_range=2
).to(device)

disc_opt = optim.Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))
gen_opt = optim.Adam(gen.parameters(), lr=lr, betas=(0.5, 0.999))

# Wasserstein produced bad result with PatchGAN
BCE_loss = nn.BCEWithLogitsLoss()
L1_loss = nn.L1Loss()

train_dataset = RetinaDataSet(root_dir=train_file_dir)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_dataset = RetinaDataSet(root_dir=val_file_dir)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
torch.cuda.empty_cache()

In [9]:
load_checkpoint("./SavedModel/c_to_nt/fixed/c_to_nt_disc_4300.pth.tar", disc, disc_opt, lr, device)
load_checkpoint("./SavedModel/c_to_nt/fixed/c_to_nt_gen_4300.pth.tar", gen, gen_opt, lr, device)

best_val = float('inf')
val_results = {}

start_epoch = 4300
for epoch in range(start_epoch, num_epochs):
    loop = tqdm(train_loader, leave=True)

    for i, (x,y) in enumerate(loop):
        nasal, temporal = y
        
        x = x.to(device).float()
        nasal = nasal.to(device).float()
        temporal = temporal.to(device).float()
        nasal_fake, temporal_fake = gen(x)

        # ------------------
        # Train Discriminator
        # ------------------
        # Evaluate on real images for nasal branch
        d_nasal_real = disc(x, nasal)
        # Evaluate on fake nasal images
        d_nasal_fake = disc(x, nasal_fake.detach())
        d_nasal_real_loss = BCE_loss(d_nasal_real, torch.ones_like(d_nasal_real))
        d_nasal_fake_loss = BCE_loss(d_nasal_fake, torch.zeros_like(d_nasal_fake))
        d_nasal_loss = (d_nasal_real_loss + d_nasal_fake_loss) / 2
        
        # Evaluate on real images for temporal branch
        d_temporal_real = disc(x, temporal)
        # Evaluate on fake temporal images
        d_temporal_fake = disc(x, temporal_fake.detach())
        d_temporal_real_loss = BCE_loss(d_temporal_real, torch.ones_like(d_temporal_real))
        d_temporal_fake_loss = BCE_loss(d_temporal_fake, torch.zeros_like(d_temporal_fake))
        d_temporal_loss = (d_temporal_real_loss + d_temporal_fake_loss) / 2
        
        # Combine discriminator losses (you could also weight these differently if needed)
        d_loss = d_nasal_loss + d_temporal_loss
        
        # ------------------
        # Train Generator
        # ------------------
        # Re-evaluate fake images with updated discriminator
        d_nasal_fake = disc(x, nasal_fake)
        d_temporal_fake = disc(x, temporal_fake)
        
        # Generator loss from fooling the discriminator
        g_nasal_fake_loss = BCE_loss(d_nasal_fake, torch.ones_like(d_nasal_fake))
        g_temporal_fake_loss = BCE_loss(d_temporal_fake, torch.ones_like(d_temporal_fake))
        g_fake_loss = g_nasal_fake_loss + g_temporal_fake_loss
        
        # L1 Loss for both outputs (to encourage similarity to ground truth)
        l1_nasal = L1_loss(nasal_fake, nasal)
        l1_temporal = L1_loss(temporal_fake, temporal)
        l1 = (l1_nasal + l1_temporal) * l1_lambda
        
        g_loss = g_fake_loss + l1
        
        gen_opt.zero_grad()
        g_loss.backward()
        gen_opt.step()

    if epoch % 10 == 0:
        best_val, val_loss = save_some_examples(gen, disc, val_loader, epoch, best_val, l1_lambda, folder="evaluation/retinal/c_to_nt")
        val_results[epoch] = val_loss.item()
        print(val_results[epoch])
    if epoch % 50 == 0 and epoch >= 3000:
        save_checkpoint(disc, disc_opt, filename="./SavedModel/c_to_nt/fixed/c_to_nt_disc_" + str(epoch) + ".pth.tar")
        save_checkpoint(gen, gen_opt, filename="./SavedModel/c_to_nt/fixed/c_to_nt_gen_" + str(epoch) + ".pth.tar")
    if epoch % 200 == 0 and epoch < 3000:
        save_checkpoint(disc, disc_opt, filename="./SavedModel/c_to_nt/fixed/c_to_nt_disc_" + str(epoch) + ".pth.tar")
        save_checkpoint(gen, gen_opt, filename="./SavedModel/c_to_nt/fixed/c_to_nt_gen_" + str(epoch) + ".pth.tar")

=> Loading checkpoint
=> Loading checkpoint


100%|███████████████████████████████████████████████████████████████████████████████████| 31/31 [00:17<00:00,  1.81it/s]


=> Saving checkpoint
=> Saving checkpoint
Epoch 4300 Validation Generator Loss: 12.9079
12.907947540283203
=> Saving checkpoint
=> Saving checkpoint


100%|███████████████████████████████████████████████████████████████████████████████████| 31/31 [00:18<00:00,  1.70it/s]


Epoch 4310 Validation Generator Loss: 17.9765
17.976451873779297


...

Epoch 5760 Validation Generator Loss: 9.3961
9.396147727966309


100%|███████████████████████████████████████████████████████████████████████████████████| 31/31 [00:17<00:00,  1.76it/s]


Epoch 5770 Validation Generator Loss: 14.0089
14.00892162322998


 52%|██████████████████████████████████████████▊                                        | 16/31 [00:09<00:08,  1.71it/s]


KeyboardInterrupt: 

In [10]:
print(val_results)

{4300: 12.907947540283203, 4310: 17.976451873779297, 4320: 18.546764373779297, 4330: 12.286130905151367, 4340: 11.124577522277832, 4350: 9.609702110290527, 4360: 17.32052230834961, 4370: 10.686064720153809, 4380: 18.982343673706055, 4390: 16.98190689086914, 4400: 12.098970413208008, 4410: 9.721601486206055, 4420: 16.962060928344727, 4430: 9.644420623779297, 4440: 10.85970401763916, 4450: 21.716384887695312, 4460: 12.447635650634766, 4470: 16.403470993041992, 4480: 23.083709716796875, 4490: 13.552044868469238, 4500: 10.671058654785156, 4510: 14.692817687988281, 4520: 9.104450225830078, 4530: 13.864392280578613, 4540: 17.657596588134766, 4550: 14.225327491760254, 4560: 14.19590950012207, 4570: 13.875663757324219, 4580: 22.525463104248047, 4590: 11.633895874023438, 4600: 16.365116119384766, 4610: 14.676251411437988, 4620: 25.526290893554688, 4630: 11.177009582519531, 4640: 14.640522003173828, 4650: 14.470771789550781, 4660: 12.663121223449707, 4670: 14.43937873840332, 4680: 20.19425582885

In [14]:
def save_test_examples(gen, disc, test_loader, folder, device):
    # Set both generator and discriminator to evaluation mode
    gen.eval()
    disc.eval()

    with torch.no_grad():
        epoch = 0
        for x,y in test_loader:
            x = x.to(device).float()
            nasal, temporal = y
            nasal.to(device).float()
            temporal.to(device).float()
            
            # Generate the fake image using the generator
            nasal_fake, temporal_fake = gen(x)
            
            # Ensure the folder exists
            os.makedirs(folder, exist_ok=True)
            os.makedirs(folder + "/fake_nasal", exist_ok=True)
            os.makedirs(folder + "/fake_temporal", exist_ok=True)
            os.makedirs(folder + "/real", exist_ok=True)
            os.makedirs(folder + "/target_nasal", exist_ok=True)
            os.makedirs(folder + "/target_temporal", exist_ok=True)
    
            # Save the images for visual comparison
            save_image(nasal_fake, os.path.join(folder + "/fake_nasal", f"fake_nasal{epoch}.png"))
            save_image(temporal_fake, os.path.join(folder + "/fake_temporal", f"fake_temporal{epoch}.png"))
            save_image(x, os.path.join(folder + "/real", f"real_{epoch}.png"))
            save_image(nasal, os.path.join(folder + "/target_nasal", f"target_nasal{epoch}.png"))
            save_image(temporal, os.path.join(folder + "/target_temporal", f"target_temporal{epoch}.png"))

            epoch += 1

    # Optionally, set the models back to training mode if needed
    gen.train()
    disc.train()

In [19]:
from pytorch_fid import fid_score

# From 4550 to 5750, every 50 epochs

test_dataset = RetinaDataSet(root_dir="./dataset/retinal/test")
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
results = []

for itera in range(4520, 4521, 50):
    print(f"Testing {itera}th...")
    load_checkpoint("./SavedModel/c_to_nt/fixed/c_to_nt_disc_" + str(itera) + ".pth.tar", disc, disc_opt, lr, device)
    load_checkpoint("./SavedModel/c_to_nt/fixed/c_to_nt_gen_" + str(itera) + ".pth.tar", gen, gen_opt, lr, device)
    save_test_examples(gen, disc, test_loader, "evaluation/retinal/c_to_nt_test_" + str(itera) + "/", device)

    # Define the paths for fake and target images
    test_output_dir = "evaluation/retinal/c_to_nt_test_" + str(itera) + "/"
    fake_path = os.path.join(test_output_dir, "fake")
    target_path = os.path.join(test_output_dir, "target")
    
    # Calculate the FID score between the fake and target image folders.
    # Adjust batch_size and dims (typically 2048 for InceptionV3) as needed.
    fid_value_nasal = fid_score.calculate_fid_given_paths([fake_path + "_nasal", target_path + "_nasal"], batch_size=50, device=device, dims=2048)
    fid_value_temporal = fid_score.calculate_fid_given_paths([fake_path + "_temporal", target_path + "_temporal"], batch_size=50, device=device, dims=2048)

    results.append({
        "Epoch": itera,
        "FID Score Nasal": fid_value_nasal,
        "FID Score Temporal": fid_value_temporal,
        "Val Loss": round(val_results[itera], 4)
    })
    print(f"Computed comparison: {results[-1]}")

Testing 4520th...
=> Loading checkpoint
=> Loading checkpoint


100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.02it/s]


Computed comparison: {'Epoch': 4520, 'FID Score Nasal': np.float64(159.8155720787399), 'FID Score Temporal': np.float64(124.41423625243425), 'Val Loss': 9.1045}


In [17]:
print(results)

[{'Epoch': 4550, 'FID Score Nasal': np.float64(157.81012886889832), 'FID Score Temporal': np.float64(124.5465908510347), 'Val Loss': 14.2253}, {'Epoch': 4600, 'FID Score Nasal': np.float64(160.83084534486346), 'FID Score Temporal': np.float64(124.6330226748218), 'Val Loss': 16.3651}, {'Epoch': 4650, 'FID Score Nasal': np.float64(159.78952695704783), 'FID Score Temporal': np.float64(123.61495706561973), 'Val Loss': 14.4708}, {'Epoch': 4700, 'FID Score Nasal': np.float64(159.92110697514357), 'FID Score Temporal': np.float64(125.04078239909445), 'Val Loss': 11.8356}, {'Epoch': 4750, 'FID Score Nasal': np.float64(160.7920922429446), 'FID Score Temporal': np.float64(123.79687462816702), 'Val Loss': 12.747}, {'Epoch': 4800, 'FID Score Nasal': np.float64(159.85242103357388), 'FID Score Temporal': np.float64(122.64551888471442), 'Val Loss': 13.7562}, {'Epoch': 4850, 'FID Score Nasal': np.float64(160.99329505614855), 'FID Score Temporal': np.float64(126.43159694575786), 'Val Loss': 11.3019}, {'

In [ ]:
load_checkpoint("./SavedModel/c_to_t/c_to_t_disc_3310.pth.tar", disc, disc_opt, lr, device)
load_checkpoint("./SavedModel/c_to_t/c_to_t_gen_3310.pth.tar", gen, gen_opt, lr, device)
print(val_loader)
for i in range(64):
    save_some_examples(gen, disc, val_loader, i, best_val, l1_lambda, folder="evaluation/retinal/c_to_t_test_3310")

In [18]:
for r in results:
    print(r)

{'Epoch': 4550, 'FID Score Nasal': np.float64(157.81012886889832), 'FID Score Temporal': np.float64(124.5465908510347), 'Val Loss': 14.2253}
{'Epoch': 4600, 'FID Score Nasal': np.float64(160.83084534486346), 'FID Score Temporal': np.float64(124.6330226748218), 'Val Loss': 16.3651}
{'Epoch': 4650, 'FID Score Nasal': np.float64(159.78952695704783), 'FID Score Temporal': np.float64(123.61495706561973), 'Val Loss': 14.4708}
{'Epoch': 4700, 'FID Score Nasal': np.float64(159.92110697514357), 'FID Score Temporal': np.float64(125.04078239909445), 'Val Loss': 11.8356}
{'Epoch': 4750, 'FID Score Nasal': np.float64(160.7920922429446), 'FID Score Temporal': np.float64(123.79687462816702), 'Val Loss': 12.747}
{'Epoch': 4800, 'FID Score Nasal': np.float64(159.85242103357388), 'FID Score Temporal': np.float64(122.64551888471442), 'Val Loss': 13.7562}
{'Epoch': 4850, 'FID Score Nasal': np.float64(160.99329505614855), 'FID Score Temporal': np.float64(126.43159694575786), 'Val Loss': 11.3019}
{'Epoch': 